In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from MuyGPyS._test.sampler import UnivariateSampler, print_results
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2
from MuyGPyS.gp.hyperparameter import AnalyticScale, Parameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize.loss import lool_fn

In [ ]:
OUTPUT_DIR = "output_gp"#"outputs"
DATASET_NAME = "Hurricane-ISABEL" 
BASE_DIR = "/lcrc/project/ECP-EZ/public/compression/aggregated-science-data"
hemant_viral_data = "/lcrc/project/SDR/zf_project"
# NUMBER_OF_LATENT_GRFS = 3 #4
NUM_CANDIDATES = 10

if DATASET_NAME == "Hurricane-ISABEL":
  T, M, N = 100, 500, 500
  SUBSETS = ["QCLOUDf48.bin", "QGRAUPf48.bin", "QICEf48.bin", "QSNOWf48.bin", "QVAPORf48.bin", "CLOUDf48.bin", "PRECIPf48.bin", "Pf48.bin", "TCf48.bin", "Uf48.bin", "Vf48.bin", "Wf48.bin"]
  SUBSET_PATH = os.path.join(BASE_DIR, f"{DATASET_NAME}/rawdata")
  GRF_PATCH_SIZE = 10
  GRF_NPATCHES = (T * (M//GRF_PATCH_SIZE) * (N//GRF_PATCH_SIZE))//1 # CHANGED from 10 to 1
  GRF_NPATCHES = int(GRF_NPATCHES)
  PRECISION = np.float32

In [ ]:
"""
  1) If 2d_grf.bin exists, load it; otherwise generate & save a new one.
  2) Run diagnostics on the loaded GRF.
"""
os.makedirs(OUTPUT_DIR, exist_ok=True)
bin_path = os.path.join(OUTPUT_DIR, "2d_grf.bin")

# --- Generate or load GRF ---
if os.path.isfile(bin_path):
    # load existing binary
    print(f"Loading existing GRF binary from {bin_path}")
    field = load_field(bin_path, config.M, config.N)
else:
    # sample a new GRF and save
    print("Binary not found; generating a new GRF.")
    grf = SpectralGaussianRandomField(
        m=config.M,
        n=config.N,
        lengthscale=config.LENGTHSCALE
    )
    field = grf.sample()
    field.astype('<f4').tofile(bin_path)
    print(f"Saved new GRF binary to {bin_path}")

# plot & save heatmap
plot_and_save_heatmap(
field=field,
save_dir=config.OUTPUT_DIR,
base_name=config.OUTPUT_DIR + f"_heatmap_ls{config.LENGTHSCALE}",
title=f"2D GRF (length scale={config.LENGTHSCALE})"
)
